# If you want to access the version you have already modified, click "Edit"
# If you want to access the original sample code, click "...", then click "Copy & Edit Notebook"

In [ ]:
_exp_name = "sample"

In [ ]:
# Import necessary packages.
import numpy as np
import torch
import os
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset

# This is for the progress bar.
from tqdm.auto import tqdm
import random
from torch.cuda import amp
from pathlib import Path
import pandas as pd

In [ ]:
myseed = 6666  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

## **Transforms**
Torchvision provides lots of useful utilities for image preprocessing, data wrapping as well as data augmentation.

Please refer to PyTorch official website for details about different transforms.

In [ ]:
# Normally, We don't need augmentations in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor.
test_tfm = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

## **Datasets**
The data is labelled by the name, so we load images and label while calling '__getitem__'

In [ ]:
class FoodDataset(Dataset):

    def __init__(self,path,tfm=test_tfm,files = None):
        super(FoodDataset).__init__()
        self.path = path
        self.files = sorted([os.path.join(path,x) for x in os.listdir(path) if x.endswith(".jpg")])
        if files != None:
            self.files = files
        print(f"One {path} sample",self.files[0])
        self.transform = tfm
  
    def __len__(self):
        return len(self.files)
  
    def __getitem__(self,idx):
        fname = self.files[idx]
        im = Image.open(fname)
        im = self.transform(im)
        #im = self.data[idx]
        try:
            label = int(fname.split("/")[-1].split("_")[0])
        except:
            label = -1 # test has no label
        return im,label


## model

In [ ]:
class Bottleneck(nn.Module):
    expansion = 4
 
    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride
 
    def forward(self, x):
        residual = x
 
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
 
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
 
        out = self.conv3(out)
        out = self.bn3(out)
 
        if self.downsample is not None:
            residual = self.downsample(x)
 
        out += residual
        out = self.relu(out)
 
        return out


class ResNet(nn.Module):
 
    def __init__(self, block, layers, num_classes=1000):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512 * block.expansion, num_classes)
 
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
 
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )
 
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))
 
        return nn.Sequential(*layers)
 
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
 
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
 
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
 
        return x


def resnet18(pretrained=False, num_classes=11):
    """Constructs a ResNet-18 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [2, 2, 2, 2], num_classes)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet18']))
    return model

## dataloader

In [ ]:
batch_size = 64
_dataset_dir = "../input/ml2022spring-hw3b/food11"

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize a model, and put it on the device specified.
model = resnet18().to(device)

## Testing and generate prediction CSV

In [ ]:
test_set = FoodDataset(os.path.join(_dataset_dir,"test"), tfm=test_tfm)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

In [ ]:
def predict(model, model_path, test_loader, tta):
    def pad4(i):
        return "0"*(4-len(str(i)))+str(i)
    
    probs = []
    prediction = []
    model.load_state_dict(torch.load(model_path))
    print(f"use pretrained model: {model_path}")
    model.eval()
    with torch.no_grad():
        for data,_ in tqdm(test_loader):
            if tta:
                data = data.to(device)
                test_pred_vertical_flip = model(torch.flip(data, dims=[2]))  # vertical flip
                test_pred_horizontal_flip = model(torch.flip(data, dims=[3]))  # horizontal flip
                test_pred = test_pred_vertical_flip * 0.5 + test_pred_horizontal_flip * 0.5  # (bs, cls_num)
            else:
                test_pred = model(data.to(device))  # (bs, cls_num)
                
            probs.append(test_pred.cpu().numpy())
            test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
            prediction += test_label.squeeze().tolist()
            
    df = pd.DataFrame()
    df["Id"] = [pad4(i) for i in range(1,len(test_set)+1)]
    df["Category"] = prediction
    if tta:
        df.to_csv(f"submission_{Path(model_path).stem}_tta.csv",index = False)
    else:
        df.to_csv(f"submission_{Path(model_path).stem}.csv",index = False)
    return np.concatenate(probs, axis=0)

## TTA

In [ ]:
model = resnet18().to(device)
model_path = "../input/emak40/ema_sample_best.ckpt"
probs = predict(model, model_path, test_loader, True)

In [ ]:
model = resnet18().to(device)
model_path = "../input/plaink40/sample_best.ckpt"
probs = predict(model, model_path, test_loader, True)

In [ ]:
def pad4(i):
    return "0"*(4-len(str(i)))+str(i)

model_pathes = ["../input/emak40/ema_sample_best.ckpt", "../input/orginal/resnet18_org2.ckpt", "../input/plaink40/sample_best.ckpt"]
probs = []
predictions = []
tta = True
for model_path in model_pathes:
    model.load_state_dict(torch.load(model_path))
    print(f"use pretrained model: {model_path}")
    tmp_probs, tmp_pred = [], []
    model.eval()
    with torch.no_grad():
        for data,_ in tqdm(test_loader):
            if tta:
                data = data.to(device)
                test_pred_vertical_flip = model(torch.flip(data, dims=[2]))  # vertical flip
                test_pred_horizontal_flip = model(torch.flip(data, dims=[3]))  # horizontal flip
                test_pred = test_pred_vertical_flip * 0.5 + test_pred_horizontal_flip * 0.5  # (bs, cls_num)
            else:
                test_pred = model(data.to(device))  # (bs, cls_num)
            
            tmp_probs.append(test_pred.cpu().numpy())
            test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
            tmp_pred += test_label.squeeze().tolist()
    probs.append(tmp_probs)
    predictions.append(tmp_pred)

### Ensemble Probalities

In [ ]:
all_probs1 = np.concatenate(probs[0], axis=0)
all_probs2 = np.concatenate(probs[1], axis=0)
all_probs3 = np.concatenate(probs[2], axis=0)
print(all_probs2.shape)

all_probs = all_probs1 * 0.33 + all_probs2 * 0.33  + all_probs3 * 0.33
all_label1 = np.argmax(all_probs, axis=1)
print(all_label1.shape)

df = pd.DataFrame()
df["Id"] = [pad4(i) for i in range(1,len(test_set)+1)]
df["Category"] = all_label1
df.to_csv(f"ensemble_prob.csv",index = False)

### Ensemble Votting

In [ ]:
len(predictions)

In [ ]:
from collections import Counter

for lab1, lab2, lab3 in zip(*predictions):
    counter = Counter([lab1, lab2, lab3])
    
    idx = list(counter.keys())
    lab = list(counter.values())